In [59]:
import glob
import numpy as np
import matplotlib.pyplot as plt

In [60]:
RG_reps = glob.glob('/media/stephen/Data/modelling-ctcf/lammps/radius_of_gyration/*-RG.txt')
mean_RG = np.mean(np.dstack([np.loadtxt(rep) for rep in RG_reps]), axis=2)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 302 and the array at index 1 has size 2202

In [ ]:
time = mean_RG[:,0]
RG = mean_RG[:,1]

In [ ]:
dydx = np.diff(RG)/ np.diff(time)
midpoints = (time[1:] + time[:-1]) / 2

In [ ]:
fig, (ax1, ax2) = plt.subplots(2,1)

ax1.set_ylabel('RG')
ax1.plot(time, RG)
ax1.tick_params(axis='y')
ax1.get_xaxis().set_visible(False)
ax1.axvline(x=1, color='Red', alpha=0.3)

ax2.set_ylabel('dRG/dt')
ax2.plot(midpoints, dydx)
ax2.tick_params(axis='y')
ax2.set_xlabel('time')
ax2.axvline(x=1, color='Red', alpha=0.3)

fig.tight_layout()
plt.show()